In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense,  GlobalAveragePooling2D, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    download=False
)

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(8).repeat(1)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128).repeat(1)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
vg19_model_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(512, 512, 3))

In [7]:
for layer in vg19_model_base.layers:
    layer.trainable = False

In [16]:
x = vg19_model_base.layers[-1].output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x= Dense(128, activation='relu')(x)
output = Dense(1, activation='softmax')(x)

In [17]:
model=Model(inputs=vg19_model_base.input, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0   

In [18]:
model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    ds_train,
    epochs=1,
    verbose = 2
)